In [1]:
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.optim as optim

from allennlp.commands.elmo import ElmoEmbedder
from collections import Counter
from datetime import datetime
from pprint import pprint
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.utils.tensorboard import SummaryWriter
from scipy.spatial.distance import cosine

In [2]:
# retrieving sentences from dataframe
df = pd.read_csv('data/corpus/corpus_processed.tsv', sep='\t', encoding='utf-8')
sentences = [s.strip() for s in df['text']]
# loading categorical and continuous emotion lexica
with open('./data/emotion_encodings/nrc_categorial.pickle', 'rb') as f:
    cat = pickle.load(f)
with open('./data/emotion_encodings/nrc_continuous.pickle', 'rb') as f:
    vad = pickle.load(f)
    
SEQ_LEN = max([len(s) for s in sentences])

In [3]:
def lex_lookup(word: str) -> np.array:
    lex_emb = [0] * 13
    if word in cat:
        lex_emb[:11] = cat[word]
    if word in vad:
        lex_emb[10:] = vad[word]
    return np.array(lex_emb)

def create_label(sentence: list) -> np.array:
    vad_emb = [e[-3:] for e in sentence]
    return sum(vad_emb)/len(vad_emb)

def embed_sentence(sentence: list) -> np.array:
    embs = elmo.embed_sentence(sentence)
    return [l for l in embs[-1]]

def create_dataset(data: list):
    corpus = []
    for sentence in tqdm(data, desc='Preparing data'):
        tokens = sentence.split()
        elmo = embed_sentence(tokens)
        emo = [lex_lookup(w) for w in tokens]
        label = create_label(emo)
        emb = [np.append(elmo[i], emo[i]) for i, _ in enumerate(elmo)]
        corpus.append((emb, label))
    with open('data/embeddings/dataset_embedded.pickle', 'wb') as d:
        pickle.dump(corpus, d)
    return corpus

def pad_sentences(array: np.array) -> np.array:
    if array.shape[0] < SEQ_LEN:
        return np.concatenate((array, np.zeros((122 - array.shape[0], array.shape[1]), dtype=np.float64)), axis=0)

In [4]:
# reading embedded dataset
with open('./data/embeddings/dataset.pickle', 'rb') as f:
    data = pickle.load(f)

In [5]:
# preparing data for pytorch
X = []
Y = []
for i in tqdm(data, desc='Padding embs'):
    X.append(pad_sentences(np.array([np.array(e, dtype=np.float64) for e in i[0]], dtype=np.float64)))
    Y.append(np.array([np.array(e, dtype=np.float64) for e in i[1]], dtype=np.float64))


X = torch.tensor(X)
Y = torch.tensor(Y)
# defining train, valid, test data
data = TensorDataset(X, Y)
train_size = int(len(data) * 0.8)
test_size = val_size = (len(data) - train_size) // 2
train, valid, test = random_split(data, [train_size, val_size, test_size])

Padding embs: 100%|██████████| 12176/12176 [00:07<00:00, 1731.17it/s]


In [6]:
# defining data-loader
BATCH_SIZE = 32
train_loader = DataLoader(train, shuffle=True, batch_size=BATCH_SIZE)
valid_loader = DataLoader(valid, shuffle=True, batch_size=BATCH_SIZE)
test_loader = DataLoader(test, shuffle=True, batch_size=BATCH_SIZE)

# testing data loader
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size())  # batch_size, seq_length
print('Sample input:\n', sample_x)
print('\nSample label size: ', sample_y.size())  # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([32, 122, 1037])
Sample input:
 tensor([[[ 0.6691, -0.1565, -0.0733,  ...,  0.6890,  0.4620,  0.6980],
         [ 0.1761,  0.0023,  0.5855,  ...,  0.6220,  0.3940,  0.7500],
         [-0.6509,  0.5465,  0.0209,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.4636,  0.2226,  0.4754,  ...,  0.9060,  0.8370,  0.8490],
         [-1.0519, -0.9248,  1.1988,  ...,  0.7500,  0.5380,  0.7040],
         [ 0.5279, -0.7262,  0.4386,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0471,  0.3201,  0.2459,  ...,  0.0000,  

In [7]:
# model architecture:
# pretrained elmo embeddings -> biLSTM -> FC -> relu + dropout -> FC -> sigmoid

class EmoRec(nn.Module):
    def __init__(self, d_emb, hidden_size, n_cells, dropout_ratio, bidirectional):
        super(EmoRec, self).__init__()
        self.n_cells = n_cells
        self.hidden_size = hidden_size
        self.out_size = 3
        self.dropout = nn.Dropout(p=dropout_ratio)
        self.rnn = nn.LSTM(input_size=sample_x.size()[-1],
                           hidden_size=hidden_size,
                           batch_first=True,
                           bidirectional=bidirectional)
        self.relu = nn.ReLU()
        self.sigm = nn.Sigmoid()

        self.fc = nn.Sequential(nn.Linear(hidden_size*2, hidden_size*4),
                                 self.relu, self.dropout,
                                 nn.Linear(hidden_size*4, hidden_size*2),
                                 self.relu, self.dropout,
                                 nn.Linear(hidden_size*2, sample_x.size()[0]))
        
        self.out = nn.Sequential(nn.Linear(sample_x.size()[0], self.out_size),
                                 self.sigm)
        
    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        state_shape = self.n_cells, batch_size, self.hidden_size
        initial_hidden_state = x.new_zeros(state_shape)
        initial_cell_state = x.new_zeros(state_shape)
        y, (ht, ct) = self.rnn(x, (initial_hidden_state, initial_cell_state))
        ht = ht[-2:].transpose(0, 1).contiguous().view(batch_size, -1)
        y = self.fc(ht) 
        y = self.out(y)
        return y


In [8]:
# CONFIG
EMB_DIM = sample_x.size()[-1]
HIDDEN_SIZE = 100
N_CELLS = 2
EPOCHS = 5

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print_every = 25
writer = SummaryWriter(f'./data/runs/e{EPOCHS}_h{HIDDEN_SIZE}')

In [9]:
model = EmoRec(EMB_DIM, HIDDEN_SIZE, N_CELLS, 0.05, True)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

In [10]:
model

EmoRec(
  (dropout): Dropout(p=0.05, inplace=False)
  (rnn): LSTM(1037, 100, batch_first=True, bidirectional=True)
  (relu): ReLU()
  (sigm): Sigmoid()
  (fc): Sequential(
    (0): Linear(in_features=200, out_features=400, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.05, inplace=False)
    (3): Linear(in_features=400, out_features=200, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.05, inplace=False)
    (6): Linear(in_features=200, out_features=32, bias=True)
  )
  (out): Sequential(
    (0): Linear(in_features=32, out_features=3, bias=True)
    (1): Sigmoid()
  )
)

In [11]:
# parameters
sum([p.numel() for p in model.parameters() if p.requires_grad])

1078331

In [12]:
# training
model.train()
model.double()
counter = 0
losses = []

for e in range(EPOCHS):
    running_loss = 0
    for x, y in train_loader:
        model.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        writer.add_scalar('train loss', loss.item(), counter)
        if counter % print_every == 0:
            val_losses = []
            model.eval()
            for x, y in valid_loader:
                out = model(x)
                val_loss = criterion(out, y)
                val_losses.append(val_loss.item())
            model.train()
            writer.add_scalar('validation loss', np.mean(val_losses), counter)
            print(f'Time: {str(datetime.now().strftime("%H:%M"))}  |  ',
                  f'Epoch: {e+1}/{EPOCHS}  |  ',
                  f'Step: {counter}  |  ',
                  f'Loss: {loss.item()}  |  ',
                  f'Val Loss: {np.mean(val_losses)}')
        counter += 1
    writer.add_scalar('batch loss', running_loss / BATCH_SIZE, e)  # epoch loss

Time: 22:25  |   Epoch: 1/5  |   Step: 0  |   Loss: 0.06681520582342886  |   Val Loss: 0.07848999690199289
Time: 22:25  |   Epoch: 1/5  |   Step: 25  |   Loss: 0.013985974419685307  |   Val Loss: 0.02499219672692923
Time: 22:25  |   Epoch: 1/5  |   Step: 50  |   Loss: 0.015552214924379646  |   Val Loss: 0.018472533005335696
Time: 22:26  |   Epoch: 1/5  |   Step: 75  |   Loss: 0.011284370747326689  |   Val Loss: 0.016262957186774292
Time: 22:26  |   Epoch: 1/5  |   Step: 100  |   Loss: 0.013882380727617471  |   Val Loss: 0.014925099380049352
Time: 22:26  |   Epoch: 1/5  |   Step: 125  |   Loss: 0.01403276306664053  |   Val Loss: 0.01270513919491932
Time: 22:26  |   Epoch: 1/5  |   Step: 150  |   Loss: 0.009468617447124966  |   Val Loss: 0.009769793533609844
Time: 22:26  |   Epoch: 1/5  |   Step: 175  |   Loss: 0.00633551839563229  |   Val Loss: 0.009198848282975585
Time: 22:26  |   Epoch: 1/5  |   Step: 200  |   Loss: 0.009913591305010896  |   Val Loss: 0.008586438122895487
Time: 22:27 

In [13]:
# testing
test_losses = []
embs = []
count = 0
model.eval()
for x, y in test_loader:
    count += 1
    out = model(x)
    embs.append(out[-1, :])
    test_loss = criterion(out, y)
    writer.add_scalar('test', test_loss.item(), count)
    test_losses.append(test_loss.item())

writer.add_embedding(torch.cat(embs, 0).view(-1, 3))


print(f'Test loss: {np.mean(test_losses)}')

Test loss: 0.0034954850024789207


In [14]:
# save torch model
torch.save(model.state_dict(), f'./data/models/e{EPOCHS}h{HIDDEN_SIZE}.pth')

In [20]:
embeddings = []
def predict(sentence: str):
    tokens = sentence.lower().split()
    elmo = embed_sentence(tokens)
    emo = [lex_lookup(w) for w in tokens]
    y = create_label(emo)
    emb = [np.append(elmo[i], emo[i]) for i, _ in enumerate(elmo)]
    with torch.no_grad():
        model.eval()
        pred = model(torch.tensor(emb).unsqueeze(dim=0))
        embeddings.append(pred[-1, :])
    with open('./data/emotion_encodings/basic_emotions.pickle', 'rb') as e:
        lookup = pickle.load(e)
    
    lookup_values = [v for v in lookup.values()]
    lookup_labels = [l for l in lookup.keys()]
    argmax = [cosine(pred, v) for v in lookup_values]
    avg = lookup_labels[np.argmax([cosine(y, v) for v in lookup_values])]
    argmax = np.argmax(argmax)
    print(f'Sentence average:\t{avg}\nPrediction:\t\t{lookup_labels[argmax]}\nSentence:\t\t{sentence}')
    return avg, lookup_labels[argmax]

In [16]:
elmo = ElmoEmbedder()

In [22]:
# testing predictions
pseudo_gold = []
prediction = []
correct = 0

for s in sentences:
    yhat, y = predict(s)
    pseudo_gold.append(yhat)
    prediction.append(y)
    if yhat == y:
        correct += 1

# adding embeddings to writer
writer.add_embedding(torch.cat(embeddings, 0).view(-1, 3))

Sentence average:	fear
Prediction:		fear
Sentence:		ill new person end lockdown
Sentence average:	fear
Prediction:		fear
Sentence:		wait blackpink going thru past three years lockdown
Sentence average:	fear
Prediction:		fear
Sentence:		things stand today capitalist civilization cannot continue must either move forward socialism fall
Sentence average:	fear
Prediction:		fear
Sentence:		use time wisely invest learn new things skill knowledge spend quality time wi
Sentence average:	fear
Prediction:		fear
Sentence:		battle nut job presidents bolsonaro wants citizens pretend covid doesnt exist duterte wants shoot
Sentence average:	fear
Prediction:		fear
Sentence:		lockdown effective way preventing spread infection already infected c
Sentence average:	fear
Prediction:		fear
Sentence:		frm mentee deal dun yday million ebitda frm mil yrly revsunderlying assets valued milbank loan mi
Sentence average:	fear
Prediction:		fear
Sentence:		active photography community even lockdown true resilience
Se

Sentence average:	fear
Prediction:		fear
Sentence:		lepad lockdown sushi date pls
Sentence average:	fear
Prediction:		fear
Sentence:		connecting route take work seen cops road stopped
Sentence average:	fear
Prediction:		fear
Sentence:		world needs bitch
Sentence average:	fear
Prediction:		fear
Sentence:		th march indian prime minister announced nationwide
Sentence average:	fear
Prediction:		fear
Sentence:		agree
Sentence average:	anger
Prediction:		fear
Sentence:		lockdown
Sentence average:	fear
Prediction:		fear
Sentence:		ldr lockdown relationship hahahahasksksjsksjkks
Sentence average:	fear
Prediction:		fear
Sentence:		thread containing nspcc warning online child abuse especially important read times
Sentence average:	fear
Prediction:		fear
Sentence:		goa customs airport staff played key role evacuation foreign tourists strandard india amidst lockdown
Sentence average:	fear
Prediction:		fear
Sentence:		new rules lockdown stay clean safe two metres away everyone
Sentence average:	dis

Sentence average:	fear
Prediction:		fear
Sentence:		home working ive put make amazing good nights sleep soul less m
Sentence average:	fear
Prediction:		fear
Sentence:		gvt shorten time limit selling essential items lockdownmay hrs morning
Sentence average:	fear
Prediction:		fear
Sentence:		lot things
Sentence average:	fear
Prediction:		fear
Sentence:		even infected person asymptomatic transfer virus many making lives miserable
Sentence average:	fear
Prediction:		fear
Sentence:		hi jemima curious expertise area vanity fair bit fashion opeds rode horses celebrity could y
Sentence average:	joy
Prediction:		fear
Sentence:		somewhere srilanka broke lockdown rules
Sentence average:	anger
Prediction:		fear
Sentence:		darrengrimes
Sentence average:	fear
Prediction:		fear
Sentence:		time
Sentence average:	fear
Prediction:		fear
Sentence:		love shades maybellinephshop hoard lockdown hihihi
Sentence average:	anger
Prediction:		fear
Sentence:		saaaame lol
Sentence average:	fear
Prediction:		fear
S

Sentence average:	fear
Prediction:		fear
Sentence:		best thing give people hours get essentials sustain week mandatory lockdown zero outdoor
Sentence average:	fear
Prediction:		fear
Sentence:		weldone priceless geng
Sentence average:	fear
Prediction:		fear
Sentence:		breaking markets open hours daily lockdown lagos ogun abuja fg
Sentence average:	fear
Prediction:		fear
Sentence:		lot traffic roads leeds
Sentence average:	fear
Prediction:		fear
Sentence:		connection issues
Sentence average:	fear
Prediction:		fear
Sentence:		india shows leadership potential fight covid pms day lockdown decision earns kudos
Sentence average:	fear
Prediction:		fear
Sentence:		coronavirus lockdown write play pandemic later coronavirus science fiction presidency rep
Sentence average:	anger
Prediction:		fear
Sentence:		lockdown
Sentence average:	fear
Prediction:		fear
Sentence:		im reading tags like set menu restaurant thank meal im read
Sentence average:	fear
Prediction:		fear
Sentence:		covid lockdown coimb

Sentence average:	anger
Prediction:		fear
Sentence:		afterwards
Sentence average:	joy
Prediction:		fear
Sentence:		kerala dumping patients toxic waste police chk mosques jamaat attendees
Sentence average:	fear
Prediction:		fear
Sentence:		lockdown relationship
Sentence average:	fear
Prediction:		fear
Sentence:		showing defiance towards ongoing lockdown proves care pay cuts loss jobs economy
Sentence average:	fear
Prediction:		fear
Sentence:		please send immediate ans strict directives heads various religions speak community m
Sentence average:	fear
Prediction:		fear
Sentence:		trump enforce state lockdowns contain coronavirus protect rights
Sentence average:	fear
Prediction:		fear
Sentence:		live apart fiancee weve lockdown weeks sent care package hones
Sentence average:	disgust
Prediction:		fear
Sentence:		whoa empty streets
Sentence average:	fear
Prediction:		fear
Sentence:		friends son vinay internationally certified nutrition amp fitness consultant hell live
Sentence average:	fear


...

Sentence average:	fear
Prediction:		fear
Sentence:		please talk
Sentence average:	fear
Prediction:		fear
Sentence:		know
Sentence average:	fear
Prediction:		fear
Sentence:		coronavirus lockdown new uk laws independent
Sentence average:	anger
Prediction:		fear
Sentence:		karna
Sentence average:	fear
Prediction:		fear
Sentence:		connecting friends lockdown everyone else facetime imessage instagram zoomme
Sentence average:	fear
Prediction:		fear
Sentence:		italys coronavirus death count still high lockdown extended april
Sentence average:	anger
Prediction:		fear
Sentence:		hours later
Sentence average:	fear
Prediction:		fear
Sentence:		amazing show wait day
Sentence average:	disgust
Prediction:		fear
Sentence:		im outside april th idc sorry
Sentence average:	fear
Prediction:		fear
Sentence:		hlo siris second lockdown plan till st may
Sentence average:	fear
Prediction:		fear
Sentence:		know number floor tiles house nwina
Sentence average:	fear
Prediction:		fear
Sentence:		lockdown stop tes

Sentence average:	fear
Prediction:		fear
Sentence:		seems entered new stage madness watched trump men last night heard donald trump say tha
Sentence average:	fear
Prediction:		fear
Sentence:		singing stops italy fear social unrest mount
Sentence average:	fear
Prediction:		fear
Sentence:		strict action taken authorities violators lockdown measures
Sentence average:	fear
Prediction:		fear
Sentence:		coronavirus lockdown excites ekiti women say blessing disguise
Sentence average:	fear
Prediction:		fear
Sentence:		lockdown finally lifted go runyou going need new running tights mom
Sentence average:	fear
Prediction:		fear
Sentence:		st kitts full lockdown
Sentence average:	anger
Prediction:		fear
Sentence:		lockdown
Sentence average:	fear
Prediction:		fear
Sentence:		simple
Sentence average:	fear
Prediction:		fear
Sentence:		kinda funny support shown babies im praying staying protecting b
Sentence average:	fear
Prediction:		fear
Sentence:		trying avoid working home lockdown
Sentence average

Sentence average:	fear
Prediction:		fear
Sentence:		see chilling lockdownoluwa wetin dey occur
Sentence average:	fear
Prediction:		fear
Sentence:		countries deadly chinese corona virus spread already people died lacs
Sentence average:	fear
Prediction:		fear
Sentence:		new independent clothing label rt
Sentence average:	fear
Prediction:		fear
Sentence:		lol authorities fighting corona virus spend big part energies towards fighting mis
Sentence average:	fear
Prediction:		fear
Sentence:		mobile testing sanigeria see mates
Sentence average:	fear
Prediction:		fear
Sentence:		morning thursday already
Sentence average:	fear
Prediction:		fear
Sentence:		families name babies corona lockdown
Sentence average:	fear
Prediction:		fear
Sentence:		fighting covid ep staying touchso youre safe youre home maybe staying safe away home lo
Sentence average:	fear
Prediction:		fear
Sentence:		every cloud smoke silver lining
Sentence average:	fear
Prediction:		fear
Sentence:		even ive already cried today able

Sentence average:	fear
Prediction:		fear
Sentence:		want donation back definitely donated earlings
Sentence average:	fear
Prediction:		fear
Sentence:		kartik aaryan sees silver lining lockdown celebrates sisters birthday together years baking c
Sentence average:	fear
Prediction:		fear
Sentence:		almost employees lost job probationary period lockdown organization fr
Sentence average:	fear
Prediction:		fear
Sentence:		incase think india far fetched nigeria oo lockdown without proper planning disaster waiting
Sentence average:	anger
Prediction:		fear
Sentence:		nigerian doctors battling
Sentence average:	fear
Prediction:		fear
Sentence:		dedicated committed tiger committed
Sentence average:	fear
Prediction:		fear
Sentence:		asking nicely vexed times
Sentence average:	fear
Prediction:		fear
Sentence:		god hunk oasis
Sentence average:	fear
Prediction:		fear
Sentence:		quarantined person maintain hygiene non quarantined one hence lockdown helps supportlockdownstaysafe
Sentence average:	fear


Sentence average:	anger
Prediction:		fear
Sentence:		lockdown ffs
Sentence average:	fear
Prediction:		fear
Sentence:		drastic decision affect many nigerians continue daily life
Sentence average:	fear
Prediction:		fear
Sentence:		life back track
Sentence average:	fear
Prediction:		fear
Sentence:		nigerian man apologizes wife learning lesson home coronavirus lockdown
Sentence average:	fear
Prediction:		fear
Sentence:		finished reading first book lockdown
Sentence average:	fear
Prediction:		fear
Sentence:		lockdown rewatching boston legal first episode already cracking absolute gold
Sentence average:	fear
Prediction:		fear
Sentence:		two masks lockdown
Sentence average:	fear
Prediction:		fear
Sentence:		know lockdown getting hand start dreaming pro clubs
Sentence average:	fear
Prediction:		fear
Sentence:		question lockdown rules
Sentence average:	fear
Prediction:		fear
Sentence:		anyone acc figures whether surfing mountain biking injuries per participant
Sentence average:	fear
Prediction:

In [23]:
pprint(Counter(pseudo_gold))
pprint(Counter(prediction))
print('Accuracy: ', (correct/len(prediction)) * 100)

Counter({'fear': 9917,
         'anger': 1479,
         'disgust': 414,
         'joy': 353,
         'surprise': 13})
Counter({'fear': 11771, 'joy': 300, 'disgust': 71, 'surprise': 34})
Accuracy:  83.45105124835742
